<a href="https://colab.research.google.com/github/nishithewel/ID5059GroupProject/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/nishithewel/ID5059GroupProject.git

Cloning into 'ID5059GroupProject'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (215/215), done.
remote: Total 265 (delta 144), reused 107 (delta 49), pack-reused 0
Receiving objects: 100% (265/265), 14.63 MiB | 11.41 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [ ]:
# import os 
# os.chdir('/content/ID5059GroupProject')
# ! git pull

### Downloading the Data from kaggle.

In [ ]:

# !pip install -q kaggle
# from google.colab import files
# files.upload()
# !rm -rf sample_data
# !mkdir  -p /root/.kaggle/
# !mv kaggle.json  /root/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json
# !ls -lart /root/.kaggle/
# !kaggle competitions download -c 'ieee-fraud-detection'
# !mkdir data
# !mv ieee-fraud-detection.zip data/
# !unzip data/ieee-fraud-detection.zip -d data/

## Introduction to the Dataset

 - Vivesh 

### Team Members
- Nishithe Welandawe - naw4
- Joseph Edwards - jde1
- Anli Hu - 373
- Yao-Ting Wang - ytw1
-


### Installs and Imports

In [ ]:
! pip install imblearn 

In [ ]:
import os

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive


os.chdir('/content/ID5059GroupProject')
from imblearn import over_sampling
from utils.preprocess import preprocess
from utils.reduce_memory import reduce_mem_usage
from sklearn.model_selection import train_test_split
from utils.model_stack import stack_models
from random import sample


### Loading the dataset

- Download the dataset from Kaggle and save it the GDrive session.
- Due to the large size, to keep loading size down we optimize data types to reduce memory usage.
- Merge the Transaction and Identity data set to create the 'Test' set.



In [ ]:
### You need to change these codes to load your dataset
#if youre files are stored in google drive do this
DRIVE_STORE = True
DEBUG = True
if DRIVE_STORE == True:
  drive.mount('/content/drive') 
  root_dir = '/content/drive/MyDrive/Python/ID5059-GroupProject/ieee-fraud-detection/' #for nish
  
else:
  root_dir = '/content/data/'
os.chdir(root_dir)

if DEBUG:
  # test_iden = pd.read_csv(root_dir + 'test_identity.csv',nrows = 100)
  # test_tran = pd.read_csv(root_dir +'test_transaction.csv',nrows = 100)
  # train_iden = pd.read_csv(root_dir +'train_identity.csv',nrows = 100)
  # train_tran = pd.read_csv(root_dir +'train_transaction.csv',nrows = 100)
  test_iden = pd.read_csv(root_dir + 'test_identity.csv',)
  test_tran = pd.read_csv(root_dir +'test_transaction.csv',)
  train_iden = pd.read_csv(root_dir +'train_identity.csv',)
  train_tran = pd.read_csv(root_dir +'train_transaction.csv',)

test_tran = reduce_mem_usage(test_tran)
train_tran = reduce_mem_usage(train_tran)  

train = pd.merge(train_tran, train_iden, on = 'TransactionID', how = 'left')
test = pd.merge(test_tran, test_iden, on = 'TransactionID', how = 'left')


Mounted at /content/drive
Memory usage of dataframe is 1519.24 MB
Memory usage after optimization is: 425.24 MB
Decreased by 72.0%
Memory usage of dataframe is 1775.15 MB


In [ ]:
test_ind = sample(range(len(train)),
                        int(0.8*len(train))
                        )
#but we need a local test to run diagnostics as well
local_test = train.iloc[test_ind]
#drop rows from train
train = train.drop(test_ind,axis = 0)

del train_tran, train_iden, test_iden, test_tran

Test_dataset and train_dataset had a mismatch in the name of id features. In the train_dataset id features were present with the name id_x where x was a value between 01 and 38 whereas in the test_dataset id features were of the form id-x. So, we changed the format of id features in the test_dataset from id-x to id_x.

In [ ]:
test.columns  = [col.replace('-','_') for col in test.columns]

## EDA
We perform basic analysis on the data.
- Visualise predictor distribution.
- Correlation


In [ ]:
train['isFraud'].value_counts(normalize = True).plot(kind= 'barh')
plt.title('% of class')
plt.ylabel('isFraud (1 = Fraud)')
plt.show()

We find very few instance of Fraud. Consider Class Balancing.

## Feature Preproccessing

Systematically, deal with features by considering the following,
- 'M' columns - look like true or false lets convert them to 1, 0
- 'D' columns - these are fine all numerical
- 'C' columns - numerical except for 4,6

We deal with categorical variables using dummy vars.

In [ ]:
train_0 = train
test_0 = test
local_test_0 = local_test

In [ ]:
# train = train_0
# test = test_0
# local_test = local_test_0

In [ ]:
train.info()

In [ ]:
train = preprocess(train)
test = preprocess(test)
local_test = preprocess(local_test)



What features are we dropping and why?

- TransactionID - Only a unique identifier for identity
- Id Columns - All Id columns contain at least 75% NaN values 


In [ ]:
id_cols = [col for col in train.columns if col.startswith('id')]
df_id = train[id_cols]
# df_id.head()
nulls = df_id.isnull().sum()/len(df_id) #as a percentage
nulls.sort_values(ascending=False).plot.bar(title='NaN as a %')
# train.drop(id_cols, axis = 1,inplace = True)

In [ ]:
#Run this otherwise might cause ram overflows
v_cols = [col for col in train.columns if col.startswith('V')]
# # lets ignore the v columns for the time being
train.drop(v_cols,axis =1, inplace = True)
train.drop(id_cols, axis = 1,inplace = True)
test.drop(v_cols,axis =1, inplace = True)
test.drop(id_cols, axis = 1,inplace = True)
local_test.drop(v_cols,axis =1, inplace = True)
local_test.drop(id_cols, axis = 1,inplace = True)

In [ ]:
categorical_features = list(set(train.columns) - set(train._get_numeric_data().columns))
#we get dummy for some features only
dummy_cols = [ col for col in categorical_features 
                    if len(train[col].unique()) < 15]
large_cats = list(set(train.columns) - set(train._get_numeric_data()) - set(dummy_cols))

In [ ]:
from sklearn.preprocessing import OneHotEncoder  #the disco import 

def dummy_transform(df,encoder,col):
  new_col = [col+'_' + str(i)  for i in encoder.categories_[0].tolist()]
  dummy_arr = encoder.transform(df[col].to_numpy().reshape(-1, 1))
  dummy_df = pd.DataFrame(dummy_arr, columns = new_col)

  # Allows nice concat-ing
  dummy_df.reset_index(drop=True, inplace=True)
  df.reset_index(drop=True, inplace=True)
  return pd.concat([df, dummy_df],axis =1)

for col in dummy_cols:
  enc = OneHotEncoder(handle_unknown='ignore',sparse=False) 
  dummy_train = enc.fit_transform(train[col].to_numpy().reshape(-1, 1))
  
  train = dummy_transform(train,enc,col)
  local_test = dummy_transform(local_test,enc,col)
  test = dummy_transform(test,enc,col)
  print('encoding',col)
  # # for df in [train, test]:
  # dummy_test = enc.transform(test[col].to_numpy().reshape(-1, 1))

  # dummy_dftest = pd.DataFrame(dummy_test, columns = new_col)
  # dummy_dftrain = pd.DataFrame(dummy_train, columns = new_col)

  # test = pd.concat([test, dummy_dftest],axis =1)
  # train = pd.concat([train, dummy_dftrain],axis =1)

    # df.drop(col,axis =1,inplace=True)
train.drop(dummy_cols,axis =1,inplace = True )
test.drop(dummy_cols,axis =1,inplace = True )
local_test.drop(dummy_cols,axis =1,inplace = True )
# train.columns

In [ ]:
train['isFraud'].unique()

In [ ]:
train.shape

We need to deal with the variables with high number of cats.

In [ ]:
def frequency_encoder(x,groupby):
  if x == None:
    return groupby.get(np.nan)
  else:
    return groupby.get(x)


  # large_cats
for col in large_cats:
  freq_enc = (train.groupby(col,dropna=False  # to prevent loss of information
                            ).size()) / len(train)
  train[col+'freq'] = train[col].apply(lambda x : frequency_encoder(x,freq_enc))
  test[col+'freq'] = train[col].apply(lambda x : frequency_encoder(x,freq_enc))
  local_test[col+'freq'] = train[col].apply(lambda x : frequency_encoder(x,freq_enc))

train.drop(large_cats,axis = 1,inplace = True)
test.drop(large_cats,axis = 1,inplace = True)
local_test.drop(large_cats,axis = 1,inplace = True)

### Column Imputation
How are we dealing with NaN/Missing/Infinite Values?

To use SMOTE we must remove np.infs and NaNs,
- So we initially drop columns with less than 80% non-NA values.
- Then, we drop rows containing NaN values.

This using performance resulted in a higher performance on 
balanced score measures than keeping NaNs and not using SMOTE.

We shoudl experiment with fillna



In [ ]:
high_na = train.columns[train.isnull().sum()/len(train)>0.2]
train_na = train.drop(high_na, axis=1)
test = test.drop(high_na, axis=1)
local_test = local_test.drop(high_na, axis=1)

In [ ]:
nulls = train_na.isnull().sum()/len(train_na) #as a percentage
nulls.sort_values(ascending=False).plot.barh(title='NaN as a %')

### Feauture Cleaning
The data contains outliers, which require cleaning.

### Feature engineering (Creation)
Wrapped as Preprocessor function to be implemented after test train split.

### Class Balancing

In [ ]:
train_no_na = train_na[~train_na.isin([np.nan, np.inf, -np.inf]).any(1)]
test = test[~local_test.isin([np.nan, np.inf, -np.inf]).any(1)]
local_test = local_test[~test.isin([np.nan, np.inf, -np.inf]).any(1)]
# train_no_na = train_no_na._get_numeric_data()
X_train, y_train = train_no_na.drop('isFraud',axis = 1), train_no_na['isFraud']
oversample = over_sampling.SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)
X_train.drop('TransactionID', axis=1, inplace = True)
X_test, y_test = local_test.drop(['isFraud', 'TransactionID'], axis = 1), local_test['isFraud']

In [ ]:
y_train.value_counts(normalize = True).plot(kind= 'barh')
plt.title('% of class')
plt.ylabel('isFraud (1 = Fraud)')
plt.show()

## Model Creation & Fitting


### Individiual models

In [ ]:

# model=CatBoostRegressor(iterations=50, depth=3, learning_rate=0.1, loss_function='RMSE')
# model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

In [ ]:
#naw4
from sklearn.model_selection import RandomizedSearchCV
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

# train.drop(['P_emaildomain', 'R_emaildomain'],axis =1 ,inplace = True)
from sklearn.metrics import accuracy_score

import xgboost as xgb

xgb_cl = xgb.XGBClassifier(
    # tree_method='gpu_hist', gpu_id=0
    )

# clf = RandomizedSearchCV(estimator=xgb_cl,
#                          param_distributions=params,
#                          n_jobs = -1,
#                         #  scoring='neg_mean_squared_error',
#                          n_iter=25,
#                          verbose=1)
# search = clf.fit(X_train, y_train)



xgb_cl.fit(X_train, y_train)
preds = xgb_cl.predict(X_test)
accuracy_score(y_train, preds)

In [ ]:
# clf.get_params()

In [ ]:
bst.save_model('model_file_name.json')

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_predictions(y_train, preds,normalize='true')

In [ ]:
#jde1
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

param_grid = [
  {'n_estimators': [10, 20, 50, 100, 500]}
 ]

bag = BaggingClassifier(max_features=0.8, max_samples=0.8)

scores = ["precision", "roc_auc"]

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    bag_clf = GridSearchCV(bag,
                           param_grid,
                           scoring=score,
                           n_jobs = -1)
    bag_clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(bag_clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = bag_clf.cv_results_["mean_test_score"]
    stds = bag_clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, bag_clf.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

In [ ]:
preds = bag_clf.predict(X_test)
accuracy_score(y_test, preds)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_train, preds,normalize='true')

In [ ]:
! pip uninstall lightgbm -y

# install lightgbm GPU
! pip install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so"


In [ ]:
#anli
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 5313,
         }
from sklearn.model_selection import train_test_split
score = 0

feature_importances = pd.DataFrame()
lgb_X_train, lgb_X_valid,lgb_y_train, lgb_y_valid=train_test_split(X_train,y_train,train_size=0.8)
dtrain = lgb.Dataset(X_train, label=y_train)
dvalid = lgb.Dataset(X_test, label=y_test)
lgb_clf = lgb.train(params, dtrain, 4000, valid_sets = [dtrain, dvalid])

feature_importances = lgb_clf.feature_importance()

y_pred_valid = lgb_clf.predict(X_test)
score = roc_auc_score(y_true, y_pred_valid)

print(f"AUC = {score}")

In [ ]:
# #wonchoi - Hyperparameter tuning for RF

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV

# # Prior to hyperparameter tuning 
# rf = RandomForestRegressor(n_estimators= 100, max_depth = 2, oob_score = True, random_state = 30000)

# # Create a parameter grid to conduct a grid search for hyperparameters
# param_grid = { 'bootstrap' : [True], 
#            'max_depth': [3, 5, 6, 10, 15, 20],
#            'n_estimators': [100, 500, 1000]}

# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 2, n_jobs = -1, verbose = 0)

# # Fit the grid search to the data and calculate its oob score
# grid_search.fit(X_train, y_train)
# grid_search.best_params_

In [ ]:
#wonchoi - RF Model

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score

# Create a random forest regressor model with the best hyperparameter fits 
rf_final = RandomForestRegressor(n_estimators = 100, max_depth = 5, n_jobs = -1, oob_score = True, random_state = 30000)
rf_final = rf_final.fit(X_train, y_train)
y_pred = rf_final.predict(X_train)
score = roc_auc_score(y_train, y_pred)

# Print AUC score
print(f"AUC = {score}")

# Calculate rmse
res_rmse = mse(y_train, y_pred, squared = True)

# Calculate oob score
res_oob = rf_final.oob_score_
print('Root mean squared error:', res_rmse,'Out of box score of final model: ', res_oob)

In [ ]:
#Import required modules
from sklearn.ensemble import AdaBoostClassifier

#Build the model
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
adaModel = ada_clf.fit(X_train, y_train)

#Adaboost Score
adaScore = adaModel.score(X_test, y_test)
print(adaScore)

### Model Ensembling

In [ ]:
stack_models(('Nishi', xgb_cl))

In [ ]:
os.chdir('/content/ID5059GroupProject')
from utils.diagnositics import get_diagnostics

In [ ]:
get_diagnostics(('xgb', xgb_cl), y_true=y_test, X=X_test)